<div style="text-align: center;">
  <h2><strong></strong></h2>
</div>

<center>

## **Prueba Técnica Novaventa**
## **Modelos**

**Autor:** Sebastian Carvalho Salazar  

</center>

# IBCF

In [ ]:
def recommend_products_ibcf(user_id, user_item_matrix, item_similarity_matrix, n_recommendations=3, k=3): 
    # Similitud coseno entre los productos
    item_similarity_df = item_similarity_matrix
    
    # Obtener los productos que el usuario ha valorado
    user_ratings = user_item_matrix.loc[user_id]
    rated_items = user_ratings[user_ratings > 0].index
    
    # Calcular la puntuación ponderada de los productos
    weighted_scores = {}
    for item in user_item_matrix.columns:
        if user_item_matrix.loc[user_id, item] == 0:  # Excluir productos que el usuario ya ha comprado
            score = 0
            similarity_sum = 0
            for rated_item in rated_items:
                similarity = item_similarity_df.loc[item, rated_item]
                score += similarity * user_item_matrix.loc[user_id, rated_item]
                similarity_sum += similarity
            if similarity_sum != 0:
                weighted_scores[item] = score / similarity_sum
    
    # Ordenar productos por la puntuación ponderada y devolver las recomendaciones
    recommended_products = sorted(weighted_scores, key=weighted_scores.get, reverse=True)
    recommended_products = recommended_products[:n_recommendations]

    if not recommended_products:
        # Si no hay recomendaciones basadas en la similitud, devolver los productos más populares
        product_popularity = user_item_matrix.sum(axis=0).sort_values(ascending=False)
        recommended_products = product_popularity.index[:n_recommendations].tolist()

    return rated_items, recommended_products

# UBCF

In [ ]:
def recommend_products_ubcf(user_id, user_item_matrix, user_item_matrix_reduced, n_recommendations=3, k=3):
    # Calcular similitud solo para el usuario elegido
    user_vector = user_item_matrix_reduced.loc[user_id].values.reshape(1, -1)
    other_users_matrix = user_item_matrix_reduced.drop(user_id).values
    similarities = cosine_similarity(sparse.csr_matrix(user_vector), sparse.csr_matrix(other_users_matrix))[0]
    similar_users = pd.Series(similarities, index=user_item_matrix.drop(user_id).index)
    
    # Seleccionar los k usuarios más similares
    top_k_users = similar_users.nlargest(k)
    
    # Calcular la puntuación ponderada de los productos
    weighted_scores = {}
    for product in user_item_matrix.columns:
        # Excluir productos que el usuario ya ha comprado
        if user_item_matrix.loc[user_id, product] == 0:
            score = 0
            similarity_sum = 0
            for similar_user in top_k_users.index:
                if user_item_matrix.loc[similar_user, product] > 0:
                    score += top_k_users[similar_user] * user_item_matrix.loc[similar_user, product]
                    similarity_sum += top_k_users[similar_user]
            if similarity_sum != 0:
                weighted_scores[product] = score / similarity_sum
    
    # Ordenar productos por la puntuación ponderada y devolver las recomendaciones
    recommended_products = sorted(weighted_scores, key=weighted_scores.get, reverse=True)
    recommended_products = recommended_products[:n_recommendations]

    if not recommended_products:
      # Si no hay recomendaciones basadas en la similitud, devolver los productos más populares
      product_popularity = user_item_matrix.sum(axis=0).sort_values(ascending=False)
      recommended_products = product_popularity.index[:n_recommendations].tolist()

    return top_k_users , recommended_products

# FIN
---